In [1]:
import math
infinity = math.inf
from itertools import chain
import numpy as np
import bisect
import copy

In [2]:
class memoize:
 # Memoizing a function makes it faster by trading space for time. 
 #    It does this by caching the return values of the function in a table. 
 #   If you call the function again with the same arguments, memoize jumps 
 #  in and gives you the value out of the table, 
 # instead of letting the function compute the value all over again.

 #. See https://www.python-course.eu/python3_memoization.php

    def __init__(self, f, memo={}):
        self.f = f
        self.memo = {}
    def __call__(self, *args):
        if not str(args) in self.memo:
            self.memo[str(args)] = self.f(*args)

        #Warning: You may wish to do a deepcopy here if returning objects
        return self.memo[str(args)]

In [3]:
# A utility function to count
# inversions in given array 'arr[]'
# See https://www.geeksforgeeks.org/check-instance-8-puzzle-solvable/ for parity check explanation
# goal = {0:[2,2], 1:[0,0], 2:[0,1], 3:[0,2], 4:[1,0], 5:[1,1], 6:[1,2], 7:[2,0], 8:[2,1]}

def coordinate(state):
    index_state = {}
    index = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1], [2,2]]
    for i in range(len(state)):
        index_state[state[i]] = index[i]
    return index_state


def getInvCount(arr):
    inv_count = 0
    empty_value = 0
    for i in range(0, 9):
        for j in range(i + 1, 9):
            if arr[j] != empty_value and arr[i] != empty_value and arr[i] > arr[j]:
                inv_count += 1
    return inv_count


# This function returns true
# if given 8 puzzle is solvable.
def isSolvable(puzzle) :

    # Count inversions in given 8 puzzle
    inv_count = getInvCount([j for sub in puzzle for j in sub])

    # return true if inversion count is even.
    return (inv_count % 2 == 0) # even invert pairs
    # return (inv_count % 2 == 1) # odd invert pairs
    
    '''
    |1|2|3|
    |8|0|4|
    |7|6|5|
    odd invert pair = (8,4), (8,5), (8,6), (8,7), (7,6), (7,5), (6,5)
    |1|2|3|
    |4|5|6|
    |7|8|0|
    even invert pair
    '''

def linear(state):
    #print(node.__repr__)
    return sum([1 if state[i] != goal[i] else 0 for i in range(9)])

@memoize
def manhattan(state):
    #index_goal = {0:[2,2], 1:[0,0], 2:[0,1], 3:[0,2], 4:[1,0], 5:[1,1], 6:[1,2], 7:[2,0], 8:[2,1]}
    index_goal = coordinate(goal)
    index_state = coordinate(state)
    
    mhd = 0
    
    for i in range(9):
        for j in range(2):
            mhd = abs(index_goal[i][j] - index_state[i][j]) + mhd
    
    return mhd

@memoize
def sqrt_manhattan(state):
    index_goal = coordinate(goal)
    index_state = coordinate(state)

    mhd = 0
    
    for i in range(9):
        for j in range(2):
            mhd = (index_goal[i][j] - index_state[i][j])**2 + mhd
    
    return math.sqrt(mhd)

@memoize
def max_heuristic(state):
    score1 = manhattan(state)
    score2 = linear(state)
    return max(score1, score2)

In [4]:
class PriorityQueueElmt:
    """ The elements of the priority queue """
    def __init__(self,val,e):
        self.val = val
        self.e = e
    
    def __lt__(self,other):
        return self.val < other.val
    
    def value(self):
        return self.val
    
    def elem(self):
        return self.e

class Queue:
    """Queue is an abstract class/interface. There are three types:
Stack(): A Last In First Out Queue.
FIFOQueue(): A First In First Out Queue.
PriorityQueue(lt): Queue where items are sorted by lt, (default <).
Each type supports the following methods and functions:
q.append(item) -- add an item to the queue
q.extend(items) -- equivalent to: for item in items: q.append(item)
q.pop() -- return the top item from the queue
len(q) -- number of items in q (also q.__len())
Note that isinstance(Stack(), Queue) is false, because we implement stacks
as lists. If Python ever gets interfaces, Queue will be an interface."""

    def __init__(self):
        pass

    def extend(self, items):
        for item in items: self.append(item)

class PriorityQueue(Queue):
    """A queue in which the minimum (or maximum) element (as determined by f and
order) is returned first. If order is min, the item with minimum f(x) is
returned first; if order is max, then it is the item with maximum f(x)."""
    def __init__(self, order=min, f=None):
        self.A=[]
        self.order=order
        self.f=f
    def append(self, item):
        queueElmt = PriorityQueueElmt(self.f(item),item)
        bisect.insort(self.A, queueElmt)
    def __len__(self):
        return len(self.A)
    def pop(self):
        if self.order == min:
            return self.A.pop(0).elem()
        else:
            return self.A.pop().elem()

In [5]:

"""
example:-
          Initial State                        Goal State
          | 7 | 2 | 4 |                       | 1 | 2 | 3 |
          | 5 | 0 | 6 |                       | 4 | 5 | 6 |
          | 8 | 3 | 1 |                       | 7 | 8 | 0 |

"""
# Heuristics for 8 Puzzle Problem
"""
Heuristics :-
1) Manhattan Distance:- For the 8 puzzle problem Manhattan distance is defined as the distance of a tile from its goal state
 (for the tile numbered '1' in the initial configuration Manhattan distance is 4 "2 for left and 2 for upward displacement").
2) No. of Misplaced Tiles:- The heuristic calculates the number of misplaced tiles between the current state and goal state.
3) Sqrt of Manhattan Distance:- It calculates the square root of Manhattan distance.
4) Max Heuristic:- It assign the score as the maximum between "Manhattan Distance" and "No. of Misplaced Tiles".
"""
    
class Problem:
    """The abstract class for a formal problem.  You should subclass this and
    implement the method inheritor, and possibly __init__, goal_test, and
    path_cost. Then you will create instances of your subclass and solve them
    with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal.  Your subclass's constructor can add
        other arguments."""
        self.initial = initial; self.goal = goal

    def inheritor(self, state):
        """Given a state, return a sequence of (action, state) pairs reachable
        from this state. If there are many inheritors, consider an iterator
        that yields the inheritors one at a time, rather than building them
        all at once. Iterators will work fine within the framework."""
        #abstract
        #here is where you have to implement the inheritor method to kake the program run
        """
          | 0 | 1 | 2 |
          | 3 | 4 | 5 |
          | 6 | 7 | 8 |
        """
        ret = []
        pos = state.index(0, 0, 9);
        if 0<=pos<=5:
            newState = state.copy()
            newState[pos], newState[pos+3] = newState[pos+3], newState[pos]
            ret.append(("UP", newState))
        if (pos%3)!=2:
            newState = state.copy()
            newState[pos], newState[pos+1] = newState[pos+1], newState[pos]
            ret.append(("LEFT", newState))
        if 3<=pos<=8:
            newState = state.copy()
            newState[pos], newState[pos-3] = newState[pos-3], newState[pos]
            ret.append(("DOWN", newState))
        if (pos%3)!=0:
            newState = state.copy()
            newState[pos], newState[pos-1] = newState[pos-1], newState[pos]
            ret.append(("RIGHT", newState))
        # print(ret)
        return (ret)

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal, as specified in the constructor. Implement this
        method if checking against a single self.goal is not enough."""
        return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        return

class Node:
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a inheritor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0, depth =0):
        "Create a search tree Node, derived from a parent by an action."

        self.parent = parent
        if parent:
            self.depth = parent.depth + 1
        else:
            self.depth = 0
        self.path_cost = path_cost
        self.state = state
        if action:
            self.action = action
        else: self.action = "init"
            
    def __repr__(self):
        #return "<Node state:\n%s > " % (np.array(self.state).reshape(3,3))
        return "Node state:\n " + str(np.array(self.state).reshape(3,3)) +"\n -> action: " + self.action + "\n -> depth: " + str(self.depth)


    def path(self):
        "Create a list of nodes from the root to this node."
        x, result = self, [self]
        while x.parent:
            result.append(x.parent)
            x = x.parent
        return result

    def expand(self, problem):
        "Return a list of nodes reachable from this node. "
        for (act,n) in problem.inheritor(self.state):
            if n not in [node.state for node in self.path()]:
                yield Node(n, self, act,
                    problem.path_cost(self.path_cost, self.state, act, n))



In [6]:
def graph_search(problem, fringe):
    """Search through the inheritors of a problem to find a goal.
    The argument fringe should be an empty queue.
    If two paths reach a state, only use the best one. [Fig. 3.18]"""
    closed = {}
    fringe.append(Node(problem.initial,depth=0))
    while fringe:
        node = fringe.pop()
        if problem.goal_test(node.state):
            return node
        if str(node.state) not in closed:
            closed[str(node.state)] = True
            fringe.extend(node.expand(problem))
    return None

def best_first_graph_search(problem, f):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have depth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    return graph_search(problem, PriorityQueue(min, f))

def astar_search(problem, h = None):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search.
    Uses the pathmax trick: f(n) = max(f(n), g(n)+h(n))."""
    h = h or problem.h
    def f(n):
        # print("f:",max(getattr(n, 'f', -infinity), n.path_cost + h(n)))
        #print("h(n):",h(n))
        #print("path_cost:",n.path_cost)
        return max(getattr(n, 'f', -infinity), n.path_cost + h(n.state))
    return best_first_graph_search(problem, f)

def print_path(path, method):
    print("*" * 30)
    print("\nPath:  (%s distance)" % method)
    for i in range(len(path)-1, -1, -1):
        print("-" * 15)
        print(path[i])
    

In [7]:
goal = [1, 2, 3, 4, 5, 6, 7, 8, 0] # original goal: even invert pair
# goal = [1, 2, 3, 8, 0, 4, 7, 6, 5] # new goal: odd invert part
# goal = [1, 2, 3, 6, 5, 4, 7, 8, 0]

# Solving the puzzle 
puzzle = [7, 2, 4, 5, 0, 6, 8, 3, 1] #original initial state
# puzzle = [5, 6, 7, 4, 0, 8, 3, 2, 1] # worst intial state (30 steps)
# puzzle = [8, 3, 5, 4, 1, 6, 2, 7, 0] # 14 steps

if(isSolvable(np.array(puzzle).reshape(3,3))):  # even true
    # checks whether the initialized configuration is solvable or not
    print("Solvable!")
    problem = Problem(puzzle,goal)
    
    path = astar_search(problem, manhattan).path()
    print_path(path, "manhattan")
    
    path = astar_search(problem, linear).path()
    print_path(path, "linear")
    
    path = astar_search(problem, sqrt_manhattan).path()
    print_path(path, "sqrt_manhattan")
    
    path = astar_search(problem, max_heuristic).path()
    print_path(path, "max_heuristic")
    
else :
    print("Not Solvable!")  # non-even false

Solvable!
******************************

Path:  (manhattan distance)
---------------
Node state:
 [[7 2 4]
 [5 0 6]
 [8 3 1]]
 -> action: init
 -> depth: 0
---------------
Node state:
 [[7 2 4]
 [5 3 6]
 [8 0 1]]
 -> action: UP
 -> depth: 1
---------------
Node state:
 [[7 2 4]
 [5 3 6]
 [8 1 0]]
 -> action: LEFT
 -> depth: 2
---------------
Node state:
 [[7 2 4]
 [5 3 0]
 [8 1 6]]
 -> action: DOWN
 -> depth: 3
---------------
Node state:
 [[7 2 4]
 [5 0 3]
 [8 1 6]]
 -> action: RIGHT
 -> depth: 4
---------------
Node state:
 [[7 2 4]
 [0 5 3]
 [8 1 6]]
 -> action: RIGHT
 -> depth: 5
---------------
Node state:
 [[0 2 4]
 [7 5 3]
 [8 1 6]]
 -> action: DOWN
 -> depth: 6
---------------
Node state:
 [[2 0 4]
 [7 5 3]
 [8 1 6]]
 -> action: LEFT
 -> depth: 7
---------------
Node state:
 [[2 4 0]
 [7 5 3]
 [8 1 6]]
 -> action: LEFT
 -> depth: 8
---------------
Node state:
 [[2 4 3]
 [7 5 0]
 [8 1 6]]
 -> action: UP
 -> depth: 9
---------------
Node state:
 [[2 4 3]
 [7 0 5]
 [8 1 6]]
 -> a